In [1]:
import numpy as np 
import pandas as pd 
import requests as re
import json
import urllib.request
import bs4 as bs

In [2]:
pip install tmdbv3api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from tmdbv3api import TMDb

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [5]:
source = urllib.request.urlopen(url).read()
scrap = bs.BeautifulSoup(source,'lxml')

In [ ]:
print(scrap.prettify())

In [7]:
tables = scrap.find_all('table', class_='wikitable sortable')

In [8]:
len(tables)

4

In [9]:
content1 = pd.read_html(str(tables[0]))[0]
content2 = pd.read_html(str(tables[1]))[0]
content3 = pd.read_html(str(tables[2]))[0]
content4 = pd.read_html(str(tables[3]))[0]

In [10]:
content = content1.append(content2.append(content3.append(content4,ignore_index=True),ignore_index=True),ignore_index=True)

In [11]:
content

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],NaN
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],NaN
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],NaN
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,NaN
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],NaN
...,...,...,...,...,...,...,...
269,DECEMBER,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,NaN,[240]
270,DECEMBER,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN,[241]
271,DECEMBER,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN,[242]
272,DECEMBER,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN,[243]


In [12]:
content.drop('Ref.', axis = 1, inplace = True)

In [13]:
content.rename(columns = {'.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.':'Ref'}, inplace = True)

In [14]:
content.shape


(274, 6)

In [15]:
content.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]


In [16]:
content_2020 = content[['Title', 'Cast and crew']]

In [17]:
tmdb = TMDb()
tmdb.api_key = '2946819469ace2bf8ccde8b306d41888'

In [18]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def source_genre(title):
      genres = []
      result = tmdb_movie.search(title)
      if not result:
        return np.NaN
      else:
        movie_id = result[0].id
        response = re.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
        else:
            np.NaN

In [21]:
content_2020['genres'] = content['Title'].map(lambda x: source_genre(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
content_2020

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama
...,...,...,...
269,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Action Fantasy Family Comedy
270,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure Action
271,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama
272,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama


In [24]:
def director(title):
  if " (director)" in title:
    return title.split(" (director)")[0]
  elif " (directors)" in title:
    return title.split(" (directors)")[0]
  else:
    return title.split(" (director/screenplay)")[0]

In [26]:
content_2020['director'] = content_2020['Cast and crew'].map(lambda x: director(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
def actor1(x):
   return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [28]:
content_2020['actor 1'] = content_2020['Cast and crew'].map(lambda x: actor1(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
def actor2(x):
   if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
   else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [30]:
content_2020['actor 2'] = content_2020['Cast and crew'].map(lambda x: actor2(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
def actor3(x):
   if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
   else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [32]:
content_2020['actor 3'] = content_2020['Cast and crew'].map(lambda x: actor3(x))

In [33]:
content_2020

,Title,Cast and crew,genres,director,actor 1,actor 2,actor 3
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
...,...,...,...,...,...,...,...
269,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Action Fantasy Family Comedy,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin
270,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure Action,Paul Greengrass,Tom Hanks,Helena Zengel,NaN
271,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge
272,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie


In [35]:
content_2020.drop('Cast and crew', axis = 1, inplace = True)

In [36]:
content_2020.head()

,Title,genres,director,actor 1,actor 2,actor 3
0,The Grudge,Horror Mystery Thriller,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Three Christs,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,Inherit the Viper,Crime Thriller Drama,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs


In [39]:
content_2020.isnull().sum()

Title        0
genres       3
director     0
actor 1      0
actor 2      5
actor 3     28
dtype: int64

In [46]:
content_2020.rename(columns = {'genres':'Genre'}, inplace = True)

In [47]:
content_2020 = content_2020.loc[:,['director','actor 1','actor 2','actor 3','Genre','Title']]

In [48]:
content_2020.head()

,director,actor 1,actor 2,actor 3,Genre,Title
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,The Grudge
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,Underwater
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,Three Christs
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,Inherit the Viper


In [49]:
content_2020.isnull().sum()

director     0
actor 1      0
actor 2      5
actor 3     28
Genre        3
Title        0
dtype: int64

In [52]:
content_2020['actor 2'] = content_2020['actor 2'].replace(np.nan, 'unknown')
content_2020['actor 3'] = content_2020['actor 3'].replace(np.nan, 'unknown')
content_2020['Genre'] = content_2020['Genre'].replace(np.nan, 'unknown')


In [53]:
content_2020

,director,actor 1,actor 2,actor 3,Genre,Title
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,The Grudge
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,Underwater
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,Three Christs
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,Inherit the Viper
...,...,...,...,...,...,...
269,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,We Can Be Heroes
270,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,Drama Western Adventure Action,News of the World
271,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,One Night in Miami...
272,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,Promising Young Woman


In [54]:
content_2020['combo'] = content_2020['actor 1'] + content_2020['actor 2'] + ' ' + content_2020['actor 3'] + ' ' + content_2020['director'] + ' ' + content_2020['Genre']

In [55]:
content_2020

,director,actor 1,actor 2,actor 3,Genre,Title,combo
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,The Grudge,Andrea RiseboroughDemián Bichir John Cho Nicol...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,Underwater,Kristen StewartVincent Cassel Jessica Henwick ...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss,Tiffany HaddishRose Byrne Salma Hayek Miguel A...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,Three Christs,Richard GerePeter Dinklage Walton Goggins Jon ...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,Inherit the Viper,Josh HartnettMargarita Levieva Chandler Riggs ...
...,...,...,...,...,...,...,...
269,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,We Can Be Heroes,Priyanka Chopra JonasPedro Pascal YaYa Gosseli...
270,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,Drama Western Adventure Action,News of the World,Tom HanksHelena Zengel unknown Paul Greengrass...
271,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,One Night in Miami...,Kingsley Ben-AdirEli Goree Aldis Hodge Regina ...
272,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,Promising Young Woman,Carey MulliganBo Burnham Alison Brie Emerald F...


In [58]:
content_17_18_19 = pd.read_csv('/content/processed_17_18_19.csv')

In [59]:
content_17_18_19

,Unnamed: 0,director,actor 1,actor 2,actor 3,Genre,Title,combo
0,0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...,...
5873,5873,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,Spies in Disguise,Will SmithTom Holland Rashida Jones Nick Bruno...
5874,5874,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,Little Women,Saoirse RonanEmma Watson Florence Pugh Greta G...
5875,5875,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action Thriller History,1917,George MacKayDean-Charles Chapman Mark Strong ...
5876,5876,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,Just Mercy,Michael B. JordanJamie Foxx Brie Larson Destin...


In [60]:
content_17_18_19.drop('Unnamed: 0', axis = 1, inplace = True)

In [61]:
content_17_18_19

,director,actor 1,actor 2,actor 3,Genre,Title,combo
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5873,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,Spies in Disguise,Will SmithTom Holland Rashida Jones Nick Bruno...
5874,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,Little Women,Saoirse RonanEmma Watson Florence Pugh Greta G...
5875,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action Thriller History,1917,George MacKayDean-Charles Chapman Mark Strong ...
5876,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,Just Mercy,Michael B. JordanJamie Foxx Brie Larson Destin...


In [62]:
processed_20 = content_17_18_19.append(content_2020, ignore_index = True)

In [63]:
processed_20

,director,actor 1,actor 2,actor 3,Genre,Title,combo
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6147,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,We Can Be Heroes,Priyanka Chopra JonasPedro Pascal YaYa Gosseli...
6148,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,Drama Western Adventure Action,News of the World,Tom HanksHelena Zengel unknown Paul Greengrass...
6149,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,One Night in Miami...,Kingsley Ben-AdirEli Goree Aldis Hodge Regina ...
6150,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,Promising Young Woman,Carey MulliganBo Burnham Alison Brie Emerald F...


In [64]:
processed_20.tail()

,director,actor 1,actor 2,actor 3,Genre,Title,combo
6147,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,We Can Be Heroes,Priyanka Chopra JonasPedro Pascal YaYa Gosseli...
6148,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,Drama Western Adventure Action,News of the World,Tom HanksHelena Zengel unknown Paul Greengrass...
6149,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,One Night in Miami...,Kingsley Ben-AdirEli Goree Aldis Hodge Regina ...
6150,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,Promising Young Woman,Carey MulliganBo Burnham Alison Brie Emerald F...
6151,Kornél Mundruczó,Vanessa Kirby,Shia LaBeouf,Molly Parker,Drama,Pieces of a Woman,Vanessa KirbyShia LaBeouf Molly Parker Kornél ...


In [69]:
processed_20.to_csv('processed_20.csv')